<a href="https://colab.research.google.com/github/YNNJN/colorPOOL_Data/blob/master/%ED%81%B4%EB%9F%AC%EC%8A%A4%ED%84%B0%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 색공간 내에서의 거리 계산
- 컬러 사이의 유사도는 색공간 내 거리와 반비례 관계
    - **Palette Similarity**
    > Inverse of distance between two palettes with the closest first matched color pairs (KAIST Color Sommelier)
- 과정
    - 테마를 구성하는 5가지 컬러 중 기준 컬러(행)와의 색차가 가장 작은(유사도가 큰) 값을 따짐

In [ ]:
# connect to google drive

from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
theme_df = pd.read_csv('gdrive/My Drive/colorPOOL/kulerData/quantizeTheme.csv')
color_df = pd.read_csv('gdrive/My Drive/colorPOOL/kulerData/quantize.csv')
referenceMatrix_df = pd.read_csv('gdrive/My Drive/colorPOOL/kulerData/referenceMatrix.csv')

In [ ]:
# inverse distance

def inverse_distance(a,b) :
  if a[0] == b[0] and a[1] == b[1] and a[2] == b[2]: # 동일한 색인 경우
    return 1.0 
  return 1/(sum((a-b)**2))**(1/2)

In [ ]:
# 컬러와 테마간의 유사도 측정
# 테마를 구성하는 5가지 컬러 중 기준 컬러(행)와 색차가 가장 작은 값을 저장함

max = 0
columns = referenceMatrix_df.columns[1:]
matrix = pd.DataFrame(columns=columns, dtype=float)

# print(len(color_df))

for colorId in range(len(color_df)):
    rowArr = []
    for themeId in range(len(theme_df)):
        color = [color_df.iloc[colorId, 1], color_df.iloc[colorId, 2], color_df.iloc[colorId, 3]]
        max_inverse_dist = 0
        for k in range(5):
            theme = [theme_df.iloc[themeId, 3*k+1], theme_df.iloc[themeId, 3*k+2], theme_df.iloc[themeId, 3*k+3]]
            if k == 0: 
                max_inverse_dist = inverse_distance(np.array(color), np.array(theme))
            else: 
                inverse_dist = inverse_distance(np.array(color), np.array(theme))
                if max_inverse_dist < inverse_dist : 
                    max_inverse_dist = inverse_dist
        if max_inverse_dist != 1 and max < max_inverse_dist :
            max = max_inverse_dist
        rowArr.append(max_inverse_dist)
    # print(rowArr) 
    matrix = matrix.append(pd.Series(rowArr,index=columns), ignore_index=True)

matrix
# print(matrix)
matrix.to_csv('gdrive/My Drive/colorPOOL/kulerData/distanceMatrix.csv')

In [ ]:
distance_df = pd.read_csv('gdrive/My Drive/colorPOOL/kulerData/distanceMatrix.csv')
theme_df = theme_df.iloc[:,1:]
color_df = color_df.iloc[:,1:]

In [5]:
theme_ids = distance_df.columns;
theme_ids = theme_ids[1:]

In [6]:
theme_df['id'] = theme_ids

In [7]:
color_df = color_df.drop_duplicates()
color_df = color_df.reset_index(drop=True)

In [8]:
copy_color_df = pd.read_csv('gdrive/My Drive/colorPOOL/kulerData/quantize.csv')
copy_color_df = copy_color_df.iloc[:,1:]
copy_color_df = copy_color_df.drop_duplicates()
copy_color_df = copy_color_df.reset_index(drop=True)

In [9]:
color_ids = []
for i in range(len(copy_color_df)) :
    color_ids.append(i)
copy_color_df['id'] = color_ids
copy_color_df

,L,a,b,id
0,36,18,30,0
1,6,3,3,1
2,72,33,81,2
3,42,69,63,3
4,99,0,36,4
...,...,...,...,...
8391,45,72,66,8391
8392,84,24,66,8392
8393,72,24,84,8393
8394,93,-15,3,8394


# 2. 클러스터링
- 과정
    - 기준 컬러(행)와 비교하여 유사도 최대값을 가지는 컬러의 범위를 그룹핑 하고, 테마 내 해당 컬러의 값을 보정함

In [ ]:
visited = [False for i in range(len(color_df))]

for r in range(len(distance_df)):
    count = 0
    L = color_df.iloc[r,0]
    a = color_df.iloc[r,1]
    b = color_df.iloc[r,2]
    print(L,a,b)

    # 해당 컬러가 이미 제거가 된 행인지 확인
    if visited[r]: continue
    visited[r] = True
  
    # 클러스터링 범위 설정
    color_range = [[L-3,a,b],[L,a-3,b],[L,a,b-3],[L+3,a,b],[L,a+3,b],[L,a,b+3]]
  
    # 클러스터링 범위 내에 컬러가 존재하는 경우
    for j in range(len(copy_color_df)): 
        for k in range(len(color_range)):
            if color_range[k][0] == copy_color_df.loc[j,'L'] and color_range[k][1] == copy_color_df.loc[j,'a'] and color_range[k][2] == copy_color_df.loc[j,'b']:
                copy_color_df.loc[j,'L'] = L
                copy_color_df.loc[j,'a'] = a
                copy_color_df.loc[j,'b'] = b

                # 방문체크
                idx = copy_color_df.loc[j,'id']
            if visited[idx] == False:  
                visited[idx] = True
                count += 1
            break

    # 중복인 컬러 제거
    copy_color_df = copy_color_df.drop_duplicates(['L','a','b'], reset_index = True)
  
    # 테마 내의 컬러 보정
    ids = []
    for theme_id in theme_ids: 
        sim = distance_df.loc[r,theme_id]
    
        if sim == 1/3: # 유사도가 최대값(1/3)일 떄
            for i in range(len(theme_df)): 
                if theme_df.loc[i,'id'] == theme_id: # 유사도 행렬의 열(theme_id)과 theme 행렬의 id가 동일하다면
                    ids.append(i)
                    for j in range(len(color_range)): # 그룹핑 진행
                        for k in range(5):
                            if color_range[j][0] == theme_df.iloc[i, 3*k] and color_range[j][1] == theme_df.iloc[i, 3*k+1] and color_range[j][2] == theme_df.iloc[i, 3*k+2]:
                                theme_df.iloc[i,3*k] = L
                                theme_df.iloc[i,3*k+1] = a
                                theme_df.iloc[i,3*k+2] = b
                                break

copy_color_df.to_csv('gdrive/My Drive/colorPOOL/kulerData/Groupquantize.csv')
theme_df.to_csv('gdrive/My Drive/colorPOOL/kulerData/GroupquantizeTheme.csv')
